In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xlsxwriter

In [2]:
df = pd.read_excel("../data/zadatak 2.xlsx", decimal = ",")
df.loc[:, "Datum"] = pd.to_datetime(df.loc[:, "Datum"])

In [3]:
fizKemPok = list(df.columns)[3:9]
rezKisika = list(df.columns)[9:13]
hranjTvar = list(df.columns)[13:22]
metali = list(df.columns)[22:24]
orgSpoj = list(df.columns)[24:26]
ioni = list(df.columns)[26:]

noviIndex = (["Pokazatelji"]
             + ["Fizikalno kemijski pokazatelji"]
             + fizKemPok
             + [""]
             + ["Režim kisika"]
             + rezKisika
             + [""]
             + ["Hranjive tvari"]
             + hranjTvar
             + [""]
             + ["Metali"]
             + metali
             + [""]
             + ["Organski spojevi"]
             + orgSpoj
             + [""]
             + ["Ioni"]
             + ioni)

In [4]:
godine = list(df.loc[:, "Datum"].apply(lambda x: x.year).unique())

In [5]:
for i in df.loc[:, "šifra"].unique():
    assert(len(df.loc[df.loc[:, "šifra"] == i, "POSTAJA"].unique()) == 1)

In [6]:
postaje = {i: df.loc[df.loc[:, "šifra"] == i, "POSTAJA"].unique(
    )[0] for i in df.loc[:, "šifra"].unique()}

In [7]:
testGodina = "2016"
testPostaja = "DA2"
testFrame = df.loc[(df.loc[:, "šifra"] == testPostaja)
                   & (df.loc[:, "Datum"].apply(lambda x: x.year).astype("str") == testGodina)]

In [8]:
def crunch(frame):
    outFrame = frame.count().to_frame()
    outFrame.columns = ["br.an."]
    outFrame.loc[:, "MAX"] = frame.max()
    outFrame.loc[:, "MIN"] = frame.min()
    outFrame.loc[:, "SR.VR."] = frame.mean()
    outFrame.loc[:, "ST.DEV."] = frame.std()
    outFrame.loc[:, "10%"] = frame.quantile(.1)
    outFrame.loc[:, "50%"] = frame.quantile(.5)
    outFrame.loc[:, "90%"] = frame.quantile(.9)
    
    return outFrame

def sliceAndDice(frame, noviIndex = noviIndex):
    outFrame = pd.DataFrame(index = pd.Index(noviIndex),
                             columns = frame.columns)
    outFrame.loc["Pokazatelji"] = frame.columns
    for i in frame.index:
        outFrame.loc[i] = frame.loc[i]
    
    return outFrame.reset_index()

In [9]:
crunchFrame = crunch(testFrame.drop(["POSTAJA", "Datum", "šifra"], axis = 1))
sliceFrame = sliceAndDice(crunchFrame)

In [11]:
writer = pd.ExcelWriter("../data/rezultat2.xlsx",
                        engine = "xlsxwriter")

counter = 0

for s, p in postaje.items():
    for g in ["2015", "2016"]:
        
        shift = counter * 53
        
        inFrame = df.loc[(df.loc[:, "šifra"] == s)
                   & (df.loc[:, "Datum"].apply(lambda x: x.year).astype("str") == str(g))]
        
        crunchFrame = crunch(inFrame.drop(["POSTAJA", "Datum", "šifra"], axis = 1))
        sliceFrame = sliceAndDice(crunchFrame)
        
        sliceFrame.to_excel(writer, sheet_name="rezultati", index=False,
                           header = False, startrow = 7 + shift)
        
        if counter == 0:
            workbook = writer.book
            workbook.strings_to_numbers = True
        
            normalFormat = workbook.add_format({"font_name":"Ariel", "font_size":10})
            subTitleFormat = workbook.add_format({"font_name":"Ariel", "font_size":10, "bold": True})
            headerFormat = workbook.add_format({"font_name":"Ariel", "font_size":14, "bold": True})

            worksheet = writer.sheets["rezultati"]
            
            worksheet.set_column(1, 8, 10, normalFormat)
            worksheet.set_column(0, 0, 40, normalFormat)

        worksheet.write(0 + shift, 0, "STATISTIKA")
        worksheet.write(1 + shift, 5, "Naziv postaje: {}".format(p))
        worksheet.write(2 + shift, 5, "Šifra postaje: {}".format(s) )
        worksheet.write(3 + shift, 5, "Razdoblje: 01.01.{}-30.12.{}".format(g, g))
        worksheet.write(4 + shift, 5, "Medij: Površinska voda")
        worksheet.write(5 + shift, 5, "Mikrolokacija:")
        worksheet.write(6 + shift, 0, "Datum ispisa: 18.10.2018")


        worksheet.set_row(0 + shift, None, headerFormat)
        worksheet.set_row(7 + shift, None, subTitleFormat)
        worksheet.set_row(8 + shift, None, subTitleFormat)
        worksheet.set_row(16 + shift, None, subTitleFormat)
        worksheet.set_row(22 + shift, None, subTitleFormat)
        worksheet.set_row(33 + shift, None, subTitleFormat)
        worksheet.set_row(37 + shift, None, subTitleFormat)
        worksheet.set_row(41 + shift, None, subTitleFormat)
        
        counter += 1
writer.save()
print("done, counter: {}".format(counter))

done, counter: 92
